In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
#
url = "https://raw.githubusercontent.com/ddw26/Team-Vand/circle_role_crkaide/output_data/df_values.csv"
catch_df = pd.read_csv(url)
catch_df.head(5)

,Date file created,Month of intercept,Time of intercept,Data sampling period,Sub region of trip,Kind of day,Fish target species,Type of gear,Mode of fishing,Mode of fishing collapsed,Collapsed area of fishing,Distance from shore,Hours on boat,Hours fished,Catch fish,Number of fish avail for inspection
0,2020-03-16,2,1512,January/February,South Atlantic (NC; SC; GA; EFL),Weekday,SPOTTED SEATROUT,hook & line,Charter boat,Charter Boat (sub_reg=6 or 7 & mode_f=7),Inland,not applicable,7.0,6.0,Yes,3
1,2020-03-16,1,1250,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,hook & line,Charter boat,Charter Boat (sub_reg=6 or 7 & mode_f=7),Inland,not applicable,5.0,4.5,No,0
2,2020-03-16,1,1250,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,hook & line,Charter boat,Charter Boat (sub_reg=6 or 7 & mode_f=7),Inland,not applicable,5.0,4.5,No,0
3,2020-03-16,2,1303,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,hook & line,Charter boat,Charter Boat (sub_reg=6 or 7 & mode_f=7),Inland,not applicable,6.0,4.0,No,0
4,2020-03-16,2,1303,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,hook & line,Charter boat,Charter Boat (sub_reg=6 or 7 & mode_f=7),Inland,not applicable,6.0,4.0,No,0


# Split the Data into Training and Testing

In [6]:
# Create our features
X = catch_df.drop('Date file created', axis=1)
X = catch_df.drop('Catch fish', axis=1)
X = pd.get_dummies(X)

# Create our target
#y = catch_df.drop('Fish target species', axis=1)
y = catch_df['Catch fish']

In [7]:
X.describe()

,Month of intercept,Time of intercept,Hours on boat,Hours fished,Number of fish avail for inspection,Date file created_2018-05-18,Date file created_2018-07-20,Date file created_2018-09-21,Date file created_2018-11-21,Date file created_2019-01-17,...,Mode of fishing_Private/Rental boat,Mode of fishing collapsed_Charter Boat (sub_reg=6 or 7 & mode_f=7),Mode of fishing collapsed_Headboat,Mode of fishing collapsed_Private/Rental Boat,Collapsed area of fishing_Inland,Collapsed area of fishing_Ocean <= 3 mi (all but WFL),Collapsed area of fishing_Ocean > 3 mi (all but WFL),Distance from shore_<= 3 miles (all but WFL),Distance from shore_> 3 miles (all but in WFL),Distance from shore_not applicable
count,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,...,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000,20238.000000
mean,7.485720,1322.732681,6.668421,4.394901,5.044619,0.021593,0.104111,0.057318,0.032612,0.012452,...,0.477369,0.226505,0.296126,0.477369,0.447574,0.255213,0.297213,0.255213,0.297213,0.447574
std,2.098378,307.940640,2.553845,1.839168,11.118904,0.145354,0.305412,0.232455,0.177623,0.110894,...,0.499500,0.418580,0.456559,0.499500,0.497256,0.435992,0.457042,0.435992,0.457042,0.497256
min,1.000000,2.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,1202.000000,5.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,1359.000000,6.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,1532.000000,8.000000,5.500000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,12.000000,2359.000000,30.000000,24.000000,300.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

Yes    11709
No      8529
Name: Catch fish, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier (n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9998276456394346

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2159,    0],
       [   1, 2900]])

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         No       1.00      1.00      1.00      1.00      1.00      1.00      2159
        Yes       1.00      1.00      1.00      1.00      1.00      1.00      2901

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      5060



In [16]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.651754137358062, 'Number of fish avail for inspection'),
 (0.046874907139768406, 'Time of intercept'),
 (0.026002639266533307, 'Hours on boat'),
 (0.0249448482253152, 'Hours fished'),
 (0.02237308465934716, 'Mode of fishing_Head boat'),
 (0.016592122940127765, 'Mode of fishing collapsed_Headboat'),
 (0.015855285368946214, 'Sub region of trip_South Atlantic (NC; SC; GA; EFL)'),
 (0.015472762610863998,
  'Sub region of trip_North Atlantic (ME; NH; MA; RI; CT)'),
 (0.013564135986063328, 'Month of intercept'),
 (0.01321007075980177,
  'Mode of fishing collapsed_Charter Boat (sub_reg=6 or 7 & mode_f=7)'),
 (0.012993895929348317, 'Mode of fishing_Charter boat'),
 (0.009554182025275025, 'Fish target species_STRIPED BASS'),
 (0.006860371725984809, 'Date file created_2020-09-22'),
 (0.006480086909471286,
  'Sub region of trip_Mid-Atlantic (NY; NJ; DE; MD; VA)'),
 (0.005931156472383285, 'Mode of fishing collapsed_Private/Rental Boat'),
 (0.005094189790144728, 'Distance from shore_<= 3 miles 

### Easy Ensemble AdaBoost Classifier

In [17]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [18]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[2159,    0],
       [   0, 2901]])

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         No       1.00      1.00      1.00      1.00      1.00      1.00      2159
        Yes       1.00      1.00      1.00      1.00      1.00      1.00      2901

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      5060

